<a href="https://colab.research.google.com/github/Jun0S2/AIBootcamp/blob/main/4%EC%A3%BC%EC%B0%A8%EA%B8%B0%EB%B3%B8%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 필요 라이브러리 설치

In [4]:
!pip install transformers datasets evaluate accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### import

In [5]:
import random
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

####  MNLI 데이터셋 로드

In [6]:
dataset = load_dataset("nyu-mll/glue", "mnli")

# Train split만 사용하여 Validation 데이터셋 생성
def create_validation_split(train_dataset, validation_ratio=0.1):
    train_split = train_dataset.train_test_split(test_size=validation_ratio, seed=42)
    return train_split["train"], train_split["test"]
train_dataset, val_dataset = create_validation_split(dataset["train"])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/52.2M [00:00<?, ?B/s]

(…)alidation_matched-00000-of-00001.parquet:   0%|          | 0.00/1.21M [00:00<?, ?B/s]

(…)dation_mismatched-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/392702 [00:00<?, ? examples/s]

Generating validation_matched split:   0%|          | 0/9815 [00:00<?, ? examples/s]

Generating validation_mismatched split:   0%|          | 0/9832 [00:00<?, ? examples/s]

Generating test_matched split:   0%|          | 0/9796 [00:00<?, ? examples/s]

Generating test_mismatched split:   0%|          | 0/9847 [00:00<?, ? examples/s]

#### Tokenizer 준비


In [7]:
# Tokenizer 준비
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)
test_dataset = dataset["validation_matched"].map(preprocess_function, batched=True)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/353431 [00:00<?, ? examples/s]

Map:   0%|          | 0/39271 [00:00<?, ? examples/s]

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

준비

In [8]:
# 데이터 Collator 준비
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 모델 준비
model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-cased", num_labels=3
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


평가 준비

In [9]:
accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir="./mnli_trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps =2,
   # num_train_epochs=3, # 속도때문에 주석
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_strategy="epoch",
    load_best_model_at_end=True,
)

# Trainer 정의
trainer = Trainer(
    model=AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-cased", num_labels=3 # 느려서 경량 모델 사용
    ),
    args=training_args,
    train_dataset=train_dataset,  # 전체 데이터셋 사용
    eval_dataset=val_dataset,    # 전체 데이터셋 사용
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-9-e692fd5b2cbe>:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


모델 학습

In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
0,1.331400,0.567230,0.768964


TrainOutput(global_step=2761, training_loss=1.3313654119318181, metrics={'train_runtime': 3044.1725, 'train_samples_per_second': 116.101, 'train_steps_per_second': 0.907, 'total_flos': 1.005089473689408e+16, 'train_loss': 1.3313654119318181, 'epoch': 0.9998189389824371})

### [Feedback]

training_args = TrainingArguments(
...
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps =4,

으로 설정 시, 예상 학습 시간 학습 시간은 약 10일 1시간 35

training_args = TrainingArguments(
...
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
으로 설정 시 , 예상 학습 시간 14일 정도로 어림잡아 계산이 되는데 뭔가 잘못 하고 있는 걸까요?
어떻게 해야 학습 시간이 과제 기간 내로는 끝날까요,,,,

평가

In [17]:
# Validation 데이터셋에 대한 성능 평가
results = trainer.evaluate(test_dataset)
print("Validation Matched Accuracy:", results["eval_accuracy"])


# 모델 저장
trainer.save_model("./mnli_model")

Validation Matched Accuracy: 0.7642384105960265


In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# 모델 및 토크나이저 로드
print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("./mnli_model")
model = AutoModelForSequenceClassification.from_pretrained("./mnli_model").to("cuda")
print("Model and tokenizer loaded successfully!")

# 테스트 예제
premise = "The sky is blue."
hypothesis = "The sky is clear."
print(f"Premise: {premise}")
print(f"Hypothesis: {hypothesis}")

# 입력 데이터 전처리
print("Tokenizing inputs...")
inputs = tokenizer(premise, hypothesis, truncation=True, padding=True, return_tensors="pt")
print("Tokenization complete.")
print(f"Tokenized inputs: {inputs}")

# token_type_ids 제거
print("Removing token_type_ids...")
inputs = {key: value.to("cuda") for key, value in inputs.items() if key != "token_type_ids"}
print("token_type_ids removed.")
print(f"Processed inputs for model: {inputs}")

# 모델 예측
print("Running model prediction...")
outputs = model(**inputs)
print("Model prediction complete.")
print(f"Model outputs (logits): {outputs.logits}")

# 예측값 추출
predictions = torch.argmax(outputs.logits, dim=1)
print(f"Raw prediction index: {predictions.item()}")

# 레이블 매핑
label_map = {0: "CONTRADICTION", 1: "NEUTRAL", 2: "ENTAILMENT"}
predicted_label = label_map[predictions.item()]
print(f"Final Prediction: {predicted_label}")

Loading model and tokenizer...
Model and tokenizer loaded successfully!
Premise: The sky is blue.
Hypothesis: The sky is clear.
Tokenizing inputs...
Tokenization complete.
Tokenized inputs: {'input_ids': tensor([[ 101, 1109, 3901, 1110, 2221,  119,  102, 1109, 3901, 1110, 2330,  119,
          102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Removing token_type_ids...
token_type_ids removed.
Processed inputs for model: {'input_ids': tensor([[ 101, 1109, 3901, 1110, 2221,  119,  102, 1109, 3901, 1110, 2330,  119,
          102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
Running model prediction...
Model prediction complete.
Model outputs (logits): tensor([[-0.1349, -1.7071,  1.8669]], device='cuda:0',
       grad_fn=<AddmmBackward0>)
Raw prediction index: 2
Final Prediction: ENTAILMENT
